In [1]:
import numpy as np
import re

from pol_analysis.polData import PolData
from pol_analysis.runPhot import RunPhot
from pol_analysis.getStokes import GetStokes

In [2]:
def run_all(obj, band, star_coords, stddev_0_vals=None, bhd_coords=(1021.5, 70.), show_seeing_plots=False, pdata_force_new=False, phot_force_new=False, special_ob_pairs=None, only_seeing_fit=False, centroid_box_size=5, pdata_filenames_for_masks=None):

    #Star coords for PSF measurements
    star_ex_ref, star_ey_ref = star_coords

    #BHD coords
    bhd_ex_ref, bhd_ey_ref = bhd_coords

    #Create the data object. This will run the mask creation and the cosmic ray rejection.  
    bhd = PolData(obj, band, filenames_for_mask_creation=pdata_filenames_for_masks, force_new=pdata_force_new)

    #Create the photometry object and run the common steps. 
    phot = RunPhot(bhd, force_new=phot_force_new)
    phot.subtract_background(box_size=(25,25))
    phot.get_source_positions(box_size=centroid_box_size)

    #Now, go through all the OB pairs.
    ob_combs = list()
    for ob_pair in bhd.ob_pairs:
        ob_combs.append([[ob_pair[0]],[ob_pair[1]]])
    if len(bhd.ob_pairs)>1:
        ob_combs.append([None, None])

    #Add special combinations if provided. 
    if special_ob_pairs is not None:
        ob_combs.extend(special_ob_pairs)

    cato = open("{}_pol_{}.dat".format(obj,  band), "w")
    for k, ob_comb in enumerate(ob_combs):
        ob_ids = ob_comb[0]
        mjds = ob_comb[1]
        #print(ob_ids, mjds)
        #continue

        #Get the seeing. 
        fnames = bhd.list_of_filenames(ob_ids=ob_ids, mjds=mjds)
        if stddev_0_vals is not None:
            stddev_0 = stddev_0_vals[k]
        else:
            stddev_0 = np.ones(len(fnames))*1.1
        phot.find_seeing(star_ex_ref, star_ey_ref, stddev_0=stddev_0, show_plots=show_seeing_plots, ob_ids=ob_ids, mjds=mjds)
        #print(phot.seeing)
        if only_seeing_fit:
            continue

        #Get the photometry convolving to a common PSF. 
        phot.get_phot(r_ap=1.0, force_new=True, apply_convolution=True, ob_ids=ob_ids, mjds=mjds)

        #Measure the stokes and polarization parameters. 
        stk_obj = GetStokes(bhd)
        stk_obj.get_pol(ob_ids=ob_ids, mjds=mjds)

        #Get the polarization values for the BHD. 
        epos_ref = np.loadtxt("{}/{}".format(bhd.phot_folder, re.sub(".fits", ".epos", fnames[0])))
        dist2 = (epos_ref[:,0]-bhd_ex_ref)**2 + (epos_ref[:,1]-bhd_ey_ref)**2
        k = np.argmin(dist2)
        #print("{:5.2f} {:5.2f} {:6.1f} {:6.1f}".format(stk_obj.pol_frac[k]*100, stk_obj.epol_frac[k]*100, stk_obj.pol_angle[k], stk_obj.epol_angle[k]))

        str = "{:5.2f} {:5.2f} {:6.1f} {:6.1f}".format(stk_obj.pol_frac[k]*100, stk_obj.epol_frac[k]*100, stk_obj.pol_angle[k], stk_obj.epol_angle[k])
        str += " {:5.1f} {:5.1f} {:5.1f}".format(np.min(phot.seeing), np.mean(phot.seeing), np.max(phot.seeing))
        if ob_ids is None:
            str += " All"
        else:
            for i, ob_id in enumerate(ob_ids):
                str += " {:10d} {:10d}".format(ob_id, mjds[i])
        print(str)
        cato.write(str+"\n")
    cato.close()

## W0019-1046

In [4]:
run_all("W0019-1046", "R_SPECIAL", star_coords=(1145, 37.5))

mkdir: work: File exists


 7.14  1.11  166.6    4.5   1.1   1.2   1.5    3565057      60201
 5.40  0.85  161.8    4.7   0.7   0.8   1.0    3565577      60202
 6.35  0.80  164.5    3.5   0.7   1.0   1.5 All


## W0204-0506

In [6]:
run_all("W0204-0506", "R_SPECIAL", star_coords=(1012.7, 53.8))

mkdir: work: File exists


25.20  1.02   12.4    1.0   0.6   0.7   0.8    3565504      60209
24.26  0.86   13.1    1.0   0.5   0.6   0.7    3565580      60207
24.72  0.64   12.7    0.8   0.5   0.7   0.8 All


## W0220+0137

In [7]:
run_all("W0220+0137", "R_SPECIAL", star_coords=(1600, 262))

mkdir: work: File exists


Finding sources for image science_reduced_img.W0220+0137.60207.R_SPECIAL.chip1.8.fits


Could not recenter o-beam position for source 53 in file science_reduced_img.W0220+0137.60201.R_SPECIAL.chip1.1.fits. Reverting to reference position.
Could not recenter o-beam position for source 81 in file science_reduced_img.W0220+0137.60201.R_SPECIAL.chip1.1.fits. Reverting to reference position.
Could not recenter o-beam position for source 81 in file science_reduced_img.W0220+0137.60201.R_SPECIAL.chip1.3.fits. Reverting to reference position.
Could not recenter o-beam position for source 53 in file science_reduced_img.W0220+0137.60201.R_SPECIAL.chip1.4.fits. Reverting to reference position.
Could not recenter o-beam position for source 53 in file science_reduced_img.W0220+0137.60201.R_SPECIAL.chip1.5.fits. Reverting to reference position.
Could not recenter o-beam position for source 81 in file science_reduced_img.W0220+0137.60201.R_SPECIAL.chip1.5.fits. Reverting to reference position.
Could not recenter o-beam position for source 55 in file science_reduced_img.W0220+0137.60201.

/Users/rjassef/miniconda3/envs/stenv/lib/python3.11/site-packages/imagematch/ImageMatching_scalerot.py:1471: RuntimeWarning: invalid value encountered in sqrt
  self.csigma = np.sqrt(self.csigma)
/Users/rjassef/miniconda3/envs/stenv/lib/python3.11/site-packages/imagematch/ImageMatching_scalerot.py:1471: RuntimeWarning: invalid value encountered in sqrt
  self.csigma = np.sqrt(self.csigma)


13.70  0.51  150.6    1.1   0.9   0.9   1.1    3565583      60201


/Users/rjassef/miniconda3/envs/stenv/lib/python3.11/site-packages/imagematch/ImageMatching_scalerot.py:1471: RuntimeWarning: invalid value encountered in sqrt
  self.csigma = np.sqrt(self.csigma)
/Users/rjassef/miniconda3/envs/stenv/lib/python3.11/site-packages/imagematch/ImageMatching_scalerot.py:1471: RuntimeWarning: invalid value encountered in sqrt
  self.csigma = np.sqrt(self.csigma)
/Users/rjassef/miniconda3/envs/stenv/lib/python3.11/site-packages/imagematch/ImageMatching_scalerot.py:1471: RuntimeWarning: invalid value encountered in sqrt
  self.csigma = np.sqrt(self.csigma)
/Users/rjassef/miniconda3/envs/stenv/lib/python3.11/site-packages/imagematch/ImageMatching_scalerot.py:1471: RuntimeWarning: invalid value encountered in sqrt
  self.csigma = np.sqrt(self.csigma)


13.60  0.44  150.2    0.9   0.5   0.6   0.7    3565623      60207
13.66  0.36  150.4    0.8   0.5   0.8   1.1 All


## W0831+0140

In [ ]:
run_all("W0831+0140", "R_SPECIAL", star_coords=(1010, 404))

## W0116-0505

#### $R_{\rm SPECIAL}$

In [ ]:
run_all("W0116-0505", "R_SPECIAL", star_coords=(1398, 413))#, only_seeing_fit=True, show_seeing_plots=True)#, centroid_box_size=5, phot_force_new=True)#, pdata_force_new=True)

#### $I_{\rm BESS}$

In [ ]:
special_ob_pairs = list()
special_ob_pairs.append([[3564862, 3565005],[60201, 60148]])
special_ob_pairs.append([[3564862, 3565005, 3564862], [60201, 60148, 60148]])

run_all("W0116-0505", "I_BESS", star_coords=(1398, 413), special_ob_pairs=special_ob_pairs)#, only_seeing_fit=True, show_seeing_plots=True)#

#### $v_{\rm HIGH}$

In [ ]:
#The images are not good for creating the masks. Instead, we will use the i-band images obtained closest in time to generate the masks. 
aux = PolData("W0116-0505",bband="I_BESS")
filenames_for_masks = aux.list_of_filenames(ob_ids=[3564862],mjds=[60146])

run_all("W0116-0505", "v_HIGH", star_coords=(1398, 413), pdata_filenames_for_masks=filenames_for_masks)#, only_seeing_fit=True, show_seeing_plots=True, phot_force_new=True, pdata_force_new=True)#